<a href="https://colab.research.google.com/github/hendfekry/my_ml_server/blob/master/ADI_Task_2(pre_processing%20Data%2Bfeature_extraction%2BML_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import nltk
import re
import string
import sys

from sklearn.neighbors import NeighborhoodComponentsAnalysis,KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline


In [3]:
df=pd.read_csv('/content/drive/MyDrive/modified_with_text.csv',lineterminator='\n',encoding='utf-8')

X = df.drop(['dialect'],axis=1).values   
y = df['dialect'].values				

# Choose test size to split between training and testing sets:
x_train, x_te, y_train, y_te = train_test_split(X, y, test_size=.7, random_state=42)


# Use the same function above for the validation set
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, 
    test_size=0.2, random_state= 42) 


In [4]:
id_dataframe=pd.DataFrame(y_train)
id_dataframe.to_csv('y_train.csv',index=False)
id_dataframe=pd.read_csv('y_train.csv')
id_dataframe.to_csv("y_train.csv", header=["dialect"], index=False)


id_dataframe=pd.DataFrame(y_test)
id_dataframe.to_csv('y_test.csv',index=False)
id_dataframe=pd.read_csv('y_test.csv')
id_dataframe.to_csv("y_test.csv", header=["dialect"], index=False)


id_dataframe=pd.DataFrame(x_train)
id_dataframe.to_csv('x_train.csv',index=False)
id_dataframe2=pd.read_csv('x_train.csv',lineterminator='\n')
id_dataframe.to_csv("x_train.csv", header=["id","text"], index=False)


id_dataframe=pd.DataFrame(x_test)
id_dataframe.to_csv('x_test.csv',index=False)
id_dataframe=pd.read_csv('x_test.csv',lineterminator='\n')
id_dataframe.to_csv("x_test.csv", header=["id","text"], index=False)



In [5]:
df=pd.read_csv('x_test.csv',lineterminator='\n')
df.head()

,id,text
0,495692014151667712,@AJABreaking اتحس ان الجبير مطرش برشامه حق بوم...
1,993436474891698176,@iiiWedad لازم يرفع للي جابه ورازه في كل مبار...
2,900498141350199296,@M_inaa___ بس ترى الكوري مقاسه سمول ماينفع لكم...
3,581242125552996352,@zanzounnaaa خرجنا منستاهل وحلال ع قلب الشاطر ...
4,751964529333039104,@Lilia_BenChikha ليليا بن شيخة انتي نجمة متالق...


In [6]:
nltk.download("stopwords")
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
train_data=pd.read_csv('x_train.csv',lineterminator='\n',encoding='utf-8')
test_data=pd.read_csv('x_test.csv',lineterminator='\n',encoding='utf-8')
y_train=pd.read_csv('y_train.csv',lineterminator='\n',encoding='utf-8')
y_test=pd.read_csv('y_test.csv',lineterminator='\n',encoding='utf-8')

train_data=train_data.drop(columns=['id'])
test_data=test_data.drop(columns=['id'])


In [8]:
x_train=np.array(train_data.text)
x_test=np.array(test_data.text)
y_train=np.array(y_train.dialect)
y_test=np.array(y_test.dialect)

In [9]:
def plot_cov_matrix(y_test,pred):
  con_mat=confusion_matrix(y_test,pred)
 # con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

  con_mat_df = pd.DataFrame(con_mat,index =np.unique(y_test), columns = np.unique(y_test))
  figure = plt.figure(figsize=(15,15))
  sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()
  print(accuracy_score(y_test,pred))
  print(f1_score(y_test,pred, average='macro'))

def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
        

In [10]:
from os import strerror
english_char=list(string.printable[:-38])
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations



def remove_links(text):
  text = re.sub(r'http\S+', '', text)
  return text

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_english(text):
    for char in english_char:
      
      text =re.sub(char, '', text)  
    return text


def remove_punctuations(text):
    
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    
    return re.sub(r'(.)\1+', r'\1', text)


def remove_stop_words(text):
        
    for word in text.split(" "):
          if word  in arb_stopwords:  
            text=text.replace(word,"")
    return text

def pre_process_document(text):
  
  text = remove_links(text)
  text = remove_punctuations(text)
  text = remove_stop_words(text)
  #text = remove_diacritics(text)
  text = remove_repeating_char(text)
  text = remove_english(text)

  return text

pre_x_train=[]  
pre_x_test=[]
for text in x_train:
 
  pre_x_train.append(pre_process_document(text))
for text in x_test:
  
  pre_x_test.append(pre_process_document(text))
pre_x_train=np.array(pre_x_train)
pre_x_test=np.array(x_test)

In [11]:

from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

estimators = [('word', TfidfVectorizer(ngram_range=(1,1),max_df=0.05,min_df=0.0001,stop_words=arb_stopwords)), ('char_b', TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char_wb',
    ngram_range=(2, 6),
    max_features=10000)),('char', TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char_wb',
    ngram_range=(2, 6),
    max_features=10000)) ]
combined = FeatureUnion(estimators)
x_train_comb = combined.fit_transform(pre_x_train, y_train)
x_test_comb=combined.transform(pre_x_test)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  % sorted(inconsistent)


In [ ]:
from sklearn import svm
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(x_train_comb, y_train)

linear_pred = linear.predict(x_test)
plot_cov_matrix(y_test,linear_pred)
accuracy_lin = linear.score(x_test, y_test)


In [ ]:
pickle.dump(linear,open('svm_model.pkl','wb'))